Q1. Write a Python program to scrape all available books from the website
(https://books.toscrape.com/) Books to Scrape – a live site built for practicing scraping (safe,
legal, no anti-bot). For each book, extract the following details:
1. Title
2. Price
3. Availability (In stock / Out of stock)
4. Star Rating (One, Two, Three, Four, Five)
Store the scraped results into a Pandas DataFrame and export them to a CSV file named
books.csv.

In [5]:
import requests
response = requests.get("https://books.toscrape.com/")
html = response.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(html,"html.parser")
books = soup.find_all("article", class_="product_pod")
title = []
price = []
avail = []
star = []
for book in books:
    title.append(book.h3.a['title'])
    p = book.find("p", class_="price_color").text
    price.append(p[1:len(p)])
    avail.append(book.find("p", class_="instock availability").text.strip())
    star.append(book.find("p", class_="star-rating")["class"][1])
    
base = "https://books.toscrape.com/catalogue/page-{}.html"
for page in range(2,51):
    url = base.format(page)
    response = requests.get(url)
    html = response.text
    
    soup = BeautifulSoup(html,"html.parser")
    books = soup.find_all("article", class_="product_pod")
    for book in books:
        title.append(book.h3.a['title'])
        p = book.find("p", class_="price_color").text
        price.append(p[1:len(p)])
        avail.append(book.find("p", class_="instock availability").text.strip())
        star.append(book.find("p", class_="star-rating")["class"][1])
     
import pandas as pd   
df = pd.DataFrame({"Title" : title, "Price" : price, "Availability": avail, "Star Rating": star})
df.to_csv("books.csv", index=False)

Q2. Write a Python program to scrape the IMDB Top 250 Movies list
(https://www.imdb.com/chart/top/) . For each movie, extract the following details:
1. Rank (1–250)
2. Movie Title
3. Year of Release
4. IMDB Rating
Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv.

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/chart/top/")
time.sleep(3)
rows = driver.find_elements(By.CLASS_NAME, "cli-children")

ranks = []
titles = []
years = []
ratings = []

for row in rows:
    ranks.append(row.find_element(By.CLASS_NAME, "ipc-title__text").text.split('.')[0].strip())
    titles.append(row.find_element(By.CLASS_NAME, "ipc-title__text").text.split('.')[1].strip())
    years.append(row.find_element(By.CLASS_NAME, "cli-title-metadata-item").text)
    ratings.append(row.find_element(By.CLASS_NAME, "ipc-rating-star--rating").text)
    
driver.quit()
df = pd.DataFrame({"Rank" : ranks, "Title" : titles, "Year": years, "Rating": ratings})
df.to_csv("imdb_top250.csv", index=False)

Q3. Write a Python program to scrape the weather information for top world cities from the
given website (https://www.timeanddate.com/weather/) . For each city, extract the following
details:
1. City Name
2. Temperature
3. Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)
Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get("https://www.timeanddate.com/weather/")

html = response.text
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table", class_="zebra")
rows = table.find_all("tr")[1:]
city = []
temp = []
weather = []

for row in rows:
    city_td = row.find("a")
    temp_td = row.find("td", class_="rbi")
    img_tag = row.find("img")
    if city_td:
        city.append(city_td.text.strip())
    else:
        city.append("N/A")
    if temp_td:
        temp.append(temp_td.text.strip())
    else:
        temp.append("N/A")
    if img_tag and "title" in img_tag.attrs:
        weather.append(img_tag["title"])
    else:
        weather.append("N/A")

df = pd.DataFrame({"City": city, "Temperature": temp, "Weather": weather})
df.to_csv("weather.csv", index=False)
